![](https://specials-images.forbesimg.com/imageserve/5e8f3beeb4aaaa0006ecf0d9/960x0.jpg?fit=scale)

<div style="background-color:powderblue;">
    <h1><center>Major League Baseball</center></h1>
</div>

A player hits a walk-off home run. A pitcher throws a no-hitter. A team gets red hot going into the Postseason. We know some of the catalysts that increase baseball fan interest. Now Major League Baseball (MLB) and Google Cloud want the Kaggle community’s help to identify the many other factors which pique supporter engagement and create deeper relationships betweens players and fans.


<div style="background-color:powderblue;">
    <h1><center>Problem statement</center></h1>
</div>

In this competition, you’ll predict how fans engage with MLB players’ digital content on a daily basis for a future date range. You’ll have access to player performance data, social media data, and team factors like market size. Successful models will provide new insights into what signals most strongly correlate with and influence engagement.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv(r'../input/mlb-player-digital-engagement-forecasting/train.csv')
teams = pd.read_csv(r'../input/mlb-player-digital-engagement-forecasting/teams.csv')
seasons = pd.read_csv(r'../input/mlb-player-digital-engagement-forecasting/seasons.csv')
players = pd.read_csv(r'../input/mlb-player-digital-engagement-forecasting/players.csv')
example_test = pd.read_csv(r'../input/mlb-player-digital-engagement-forecasting/example_test.csv')
sample_submission = pd.read_csv(r'../input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv')
awards = pd.read_csv(r'../input/mlb-player-digital-engagement-forecasting/awards.csv')


<div style="background-color:powderblue;">
    <h1><center>train & test</center></h1>
</div>

Refrence: [https://www.kaggle.com/chumajin/eda-of-mlb-for-starter-english-ver](https://www.kaggle.com/chumajin/eda-of-mlb-for-starter-english-ver)

do check him out!

In [ ]:
example_test.head()

It seems that various information is included in json format.

The test data consists of one line of data per day.


For example, using a function from the starter code, it can be expanded as follows.

In [ ]:
# Helper function to unpack json found in daily data
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

Visualize the example_test["games"].iloc[0]

In [ ]:
unpack_json(example_test["games"].iloc[0])

Visualize the example_test["rosters"].iloc[0]

In [ ]:
unpack_json(example_test["rosters"].iloc[0])

 Based on the above, we will look at other data such as train data

In [ ]:
train.head()

In [ ]:
#date is converted to datetime
train['date'] = pd.to_datetime(train['date'], format="%Y%m%d")

#### From here Let's take a look at one json where there is data for each column.
As mentioned above, the contents of train.csv are also in json file format in one cell, and it has a complicated shape like dataframe is further contained.

(According to the result, it is an image that about 11 DataFrames (although it is a little small even if there is nan) are hanging as the amount of information for one day of data for 1216 days in total, which is a considerable amount of information. is.

So it's a little long from here. You may just grab the image and read it through. )

In [ ]:
train.columns

It is troublesome to input one by one, so I will omit NaN and make the nth (0 is the top) sample as a dataframe and create a function to see the column name and contents.

In [ ]:
def exshow(col,n):
    tmp = train[col]
    tmp = tmp.dropna()
    tmpdf = unpack_json(tmp.iloc[n])
    print(tmpdf.columns)
    return tmpdf


<div style="background-color:powderblue;">
    <h1><center>1.1 nextDayPlayerEngagement (column 1 of train.csv)</center></h1>
</div>
Nested JSON containing all modeling targets from the following day.

In [ ]:
tmpdf = exshow("nextDayPlayerEngagement",0)
tmpdf.head()

* engagementMetricsDate - date of player engagement metrics, based on US Pacific Time (aligns with previous day’s games, rosters, on-field statistics, transactions, awards, etc.).
* playerId
* target1
* target2
* target3
* target4


target1-target4 are each daily indexes of digital engagement on a 0-100 scale.

From here, plyaerId and targets1 to 4 from the next day onwards are pulled out.


<div style="background-color:powderblue;">
    <h1><center>1.2 games(column 2 of train.csv)</center></h1>
</div>
Nested JSON containing all game information for a given day. Includes spring training and exhibition games along with regular season, Postseason, and All-Star games.

In [ ]:
tmpdf = exshow("games",1) 
tmpdf.head(3)

Please open ↓ for the meaning of the column. (Because it is long, I hide it.)

* gamePk - Unique identifier for a game.
* gameType - Game type, various types can be found here.
* season
* gameDate
* gameTimeUTC - First pitch time in UTC.
* resumeDate - Time game was resumed (if abandoned, otherwise null).
* resumedFrom - Time game was originally abandoned (if abandoned, otherwise null).
* codedGameState - Game status code, various types can be found here.
* detailedGameState - Game status, various types can be found here.
* isTie - Boolean, true if game ended in a tie.
* gameNumber - Game number flag to distinguish double headers - 1 or 2.
* doubleHeader - Y for DH, N for single game, S for split-ticket.
* dayNight - Day or night flag for scheduled start time.
* scheduledInnings - Scheduled number of innings.
* gamesInSeries - Number of games in current series.
* seriesDescription - Text description of current series.
* homeId - Unique identifier for home team.
* homeName - Home team name.
* homeAbbrev - Home team abbreviation.
* homeWins - Number of current wins on season for home team.
* homeLosses - Number of current losses on season for home team.
* homeWinPct - Current winning percentage for home team.
* homeWinner - Boolean, true if home team wins.
* homeScore - Runs scored by home team.
* awayId - Unique identifier for away team.
* awayName - Unique identifier for away team.
* awayAbbrev - Away team abbreviation.
* awayWins - Number of current wins on season for away team.
* awayLosses - Number of current losses on season for away team.
* awayWinPct - Current winning percentage for away team.
* awayWinner - Boolean, true if away team wins.
* awayScore - Runs scored by away team.

 
<div style="background-color:powderblue;">
    <h1><center>1.3 rosters(column 3 of train.csv)</center></h1>
</div>
Nested JSON that contains all the roster information for a particular day. Includes in-season and off-season team rosters.

In [ ]:
tmpdf = exshow("rosters",0) 
tmpdf.head()

* playerId - Unique identifier for a player.
* gameDate
* teamId - teamId that player is on that date.
* statusCode - Roster status abbreviation.
* status - Descriptive roster status.

<div style="background-color:powderblue;">
    <h1><center>1.4 playerBoxScores(column 4 of train.csv)</center></h1>
</div>
Nested JSON containing game stats aggregated at the player game level for a given day. Includes regular

In [ ]:
tmpdf = exshow("playerBoxScores",0) 
tmpdf.head(3)

Please open ↓ for the meaning of the column. (Because it is long, I hide it.)

* home - Binary, 1 if home team, 0 if away.
* gamePk - unique identifier for game.
* gameDate
* gameTimeUTC - First pitch time in UTC.
* teamId - Unique identifier for team.
* teamName
* playerId - Unique identifier for player.
* playerName
* jerseyNum
* positionCode - Number position code, details are here.
* positionName - Text position display, details are here.
* positionType - Position group, details are here.
* battingOrder - Format: “###”, where the first digit indicates the batting order spot, and the second two digits indicate the sequence in which that player occupied that batting order spot. Examples: “300”, which indicates the starter in the third spot in the batting order; “903,” which indicates the fourth player (after 900, 901 and 902) occupy the ninth spot in the batting order. Only populates if appeared in game.
* gamesPlayedBatting - 1 if player entered the game as a batter, runner or fielder.
* flyOuts - Game total fly outs.
* groundOuts - Game total ground outs.
* runsScored - Game total runs scored.
* doubles - Game total doubles.
* triples - Game total triples.
* homeRuns - Game total home runs.
* strikeOuts - Game total strike outs.
* baseOnBalls - Game total walks.
* intentionalWalks - Game total intentional walks.
* hits - Game total hits.
* hitByPitch - Game total hit by pitches.
* atBats - Game total at-bats.
* caughtStealing - Game total caught stealing.
* stolenBases - Game total stolen bases.
* groundIntoDoublePlay - Game total double plays grounded into.
* groundIntoTriplePlay - Game total triple plays grounded into.
* plateAppearances - Game total plate appearances.
* totalBases - Game total bases.
* rbi - Game total runs batted in.
* leftOnBase - Game total runners left on base.
* sacBunts - Game total sacrifice bunts.
* sacFlies - Game total sacrifice flies.
* catchersInterference - Game total catchers interference reached on.
* pickoffs - Game total number of times picked off base.
* gamesPlayedPitching- Binary, 1 if player entered the game as a pitcher.
* gamesStartedPitching- Binary, 1 if player was game’s starting pitcher.
* completeGamesPitching - Binary, 1 if credited with complete game.
* shutoutsPitching - Binary, 1 if credited with shutout.
* winsPitching - Binary, 1 if credited with win.
* lossesPitching - Binary, 1 if credited with loss.
* flyOutsPitching - Game total of flyouts allowed.
* airOutsPitching - Game total of air outs (flyouts + popouts) allowed.
* groundOutsPitching - Game total ground outs allowed.
* runsPitching - Game total runs allowed.
* doublesPitching - Game total doubles allowed.
* triplesPitching - Game total triples allowed.
* homeRunsPitching - Game total home runs allowed.
* strikeOutsPitching - Game total strike outs allowed.
* baseOnBallsPitching - Game total walks allowed.
* intentionalWalksPitching - Game total intentional walks allowed.
* hitsPitching - Game total hits allowed.
* hitByPitchPitching - Game total hit by pitches allowed.
* atBatsPitching - Game total at-bats against.
* caughtStealingPitching - Game total caught stealing against.
* stolenBasesPitching - Game total stolen bases allowed.
* inningsPitched - Game total innings pitched.
* saveOpportunities - Binary, 1 if credited with save opportunity.
* earnedRuns - Game total earned runs allowed.
* battersFaced - Game total batters faced.
* outsPitching - Game total outs recorded.
* pitchesThrown - Game total number of pitches thrown.
* balls - Game total balls thrown.
* strikes - Game total strikes thrown.
* hitBatsmen - Game total hit batters.
* balks - Game total balks.
* wildPitches - Game total number of wild pitches thrown.
* pickoffsPitching - Game total number of pickoffs.
* rbiPitching - Game total number of runs batted in allowed.
* inheritedRunners - Game total of inherited runners assumed.
* inheritedRunnersScored- Game total of inherited runners that scored.
* catchersInterferencePitching - Game total of catcher’s interference occurred by battery.
* sacBuntsPitching - Game total sacrifice bunts allowed.
* sacFliesPitching - Game total sacrifice flies allowed.
* saves - Binary, 1 if credited with save.
* holds - Binary, 1 if credited with hold.
* blownSaves - Binary, 1 if credited with blown save.
* assists - Game total number of assists.
* putOuts - Game total number of putouts.
* errors - Game total number of errors.
* chances - Game total fielding chances.


<div style="background-color:powderblue;">
    <h1><center>1.5 teamBoxScores(column 5 of train.csv)</center></h1>
</div>
Nested JSON containing game stats aggregated at the team game level for a given day. Includes regular season, Postseason, and All-Star games.

In [ ]:
tmpdf = exshow("teamBoxScores",0) 
tmpdf.head(3)

Please open ↓ for the meaning of the column. (Because it is long, I hide it.)

* home - Binary, 1 if home team, 0 if away.
* teamId - unique identifier for team.
* gamePk - unique identifier for game.
* gameDate
* gameTimeUTC - First pitch time in UTC.
* flyOuts - Game total fly outs.
* groundOuts - Game total ground outs.
* runsScored - Game total runs scored.
* doubles - Game total doubles.
* triples - Game total triples.
* homeRuns - Game total home runs.
* strikeOuts - Game total strike outs.
* baseOnBalls - Game total walks.
* intentionalWalks - Game total intentional walks.
* hits - Game total hits.
* hitByPitch - Game total hit by pitches.
* atBats - Game total at-bats.
* caughtStealing - Game total caught stealing.
* stolenBases - Game total stolen bases.
* groundIntoDoublePlay - Game total double plays grounded into.
* groundIntoTriplePlay - Game total triple plays grounded into.
* plateAppearances - Game total plate appearances.
* totalBases - Game total bases.
* rbi - Game total runs batted in.
* leftOnBase - Game total runners left on base.
* sacBunts - Game total sacrifice bunts.
* sacFlies - Game total sacrifice flies.
* catchersInterference - Game total catcher’s interference reached on.
* pickoffs - Game total number of times picked off base.
* airOutsPitching - Game total of air outs (flyouts + popouts) allowed.
* groundOutsPitching - Game total ground outs allowed.
* runsPitching - Game total runs allowed.
* doublesPitching - Game total doubles allowed.
* triplesPitching - Game total triples allowed.
* homeRunsPitching - Game total home runs allowed.
* strikeOutsPitching - Game total strike outs allowed.
* baseOnBallsPitching - Game total walks allowed.
* intentionalWalksPitching - Game total intentional walks allowed.
* hitsPitching - Game total hits allowed.
* hitByPitchPitching - Game total hit by pitches allowed.
* atBatsPitching - Game total at-bats against.
* caughtStealingPitching - Game total caught stealing against.
* stolenBasesPitching - Game total stolen bases allowed.
* inningsPitched - Game total innings pitched.
* earnedRuns - Game total earned runs allowed.
* battersFaced - Game total batters faced.
* outsPitching - Game total outs recorded.
* hitBatsmen - Game total hit batters.
* balks - Game total balks.
* wildPitches - Game total number of wild pitches thrown.
* pickoffsPitching - Game total number of pickoffs.
* rbiPitching - Game total number of runs batted in allowed.
* inheritedRunners - Game total of inherited runners assumed.
* inheritedRunnersScored- Game total of inherited runners that scored.
* catchersInterferencePitching - Game total of catcher’s interference occurred by battery.
* sacBuntsPitching - Game total sacrifice bunts allowed.
* sacFliesPitching - Game total sacrifice flies allowed.

 
<div style="background-color:powderblue;">
    <h1><center>1.6 transactions(column 6 of train.csv)</center></h1>
</div>
Nested JSON containing all transaction information involving MLB teams for a given day.

In [ ]:
tmpdf = exshow("transactions",1) 
tmpdf.head(3)

* transactionId - Unique identifier for a transaction.
* playerId - Unique identifier for a player.
* playerName
* date
* fromTeamId - Unique identifier for the team a player goes from.
* fromTeamName
* toTeamId - Unique identifier for the team a player goes to.
* toTeamName
* effectiveDate
* resolutionDate
* typeCode - Transaction status abbreviation.
* typeDesc - Transaction status description.
* description - Text description of transaction.


<div style="background-color:powderblue;">
    <h1><center>1.7 standings(column 7 of train.csv)</center></h1>
</div>
Nested JSON containing all standings information involving MLB teams for a given day.

In [ ]:
tmpdf = exshow("standings",0) 
tmpdf.head(3)

Please open ↓ for the meaning of the column. (Because it is long, I hide it.)

* season
* gameDate
* divisionId - Unique identifier of the division that this team plays in.
* teamId - Unique identifier for a team.
* teamName
* streakCode - Abbreviation of team’s current win or loss streak. First letter indicates winning or losing, number is the number of games.
* divisionRank - Team’s current rank in their division.
* leagueRank - Team’s current rank in their league.
* wildCardRank - Team’s current rank for a wildcard berth.
* leagueGamesBack - Games back in team’s league.
* sportGamesBack - Games back in all of MLB.
* divisionGamesBack - Games back in team’s division.
* wins - Current wins.
* losses - Current losses.
* pct - Current winning percentage.
* runsAllowed - Runs allowed on the season.
* runsScored - Runs scored on the season.
* divisionChamp - true if team has clinched the division title.
* divisionLeader - true if team is leading the division race.
* wildCardLeader - true if team is wildcard leader.
* eliminationNumber - number of games (team losses + opponent wins) before eliminated from the division race.
* wildCardEliminationNumber - number of games (team losses + opponent wins) before eliminated from the wildcard race.
* homeWins - Home wins on the season.
* homeLosses - Home losses on the season.
* awayWins - Away wins on the season.
* awayLosses - Away losses on the season.
* lastTenWins - Wins in the last 10 games.
* lastTenLosses - Losses in the last 10 games.
* extraInningWins - Wins in extra innings on the season.
* extraInningLosses - Losses in extra innings on the season.
* oneRunWins - Wins by one run on the season.
* oneRunLosses - Losses by one run on the season.
* dayWins - Day game wins on the season.
* dayLosses Day game losses on the season.
* nightWins - Night game wins on the season.
* nightLosses - Night game losses on the season.
* grassWins - Grass field wins on the season.
* grassLosses - Grass field losses on the season.
* turfWins - Turf field wins on the season.
* turfLosses - Turf field losses on the season.
* divWins - Wins against division opponents on the season.
* divLosses - Losses against division opponents on the season.
* alWins - Wins against AL teams on the season.
* alLosses - Losses against AL teams on the season.
* nlWins - Wins against NL teams on the season.
* nlLosses - Losses against NL teams on the season.
* xWinLossPct - Expected winning percentage based on runs scored and allowed.


<div style="background-color:powderblue;">
    <h1><center>1.8 awards(column 8 of train.csv)</center></h1>
</div>
Nested JSON containing all awards or honors handed out on a given day.

In [ ]:
tmpdf = exshow("awards",0) 
tmpdf

* awardId
* awardName
* awardDate - Date award was given.
* awardSeason - Season award was from.
* playerId - Unique identifier for a player.
* playerName
* awardPlayerTeamId


<div style="background-color:powderblue;">
    <h1><center>1.9 events(column 9 of train.csv)</center></h1>
</div>
Nested JSON containing all on-field game events for a given day. Includes regular season and Postseason games.

In [ ]:
tmpdf = exshow("events",0) 
tmpdf.head(3)

Please open ↓ for the meaning of the column. (Because it is long, I hide it.)

* gamePk - unique identifier for game.
* gameDate
* gameTimeUTC - First pitch time in UTC.
* season
* gameType - Game type, various types can be found here.
* playId - Statcast play guid.
* eventId
* inning - Inning AB occurred.
* halfInning- “top” or “bottom” inning indicator.
* homeScore - Home score at start of the event.
* awayScore - Away score at start of the event.
* menOnBase - Split used if runner on base – ie (RISP, Empty).
* atBatIndex - At-Bat index number, begins with 0.
* atBatDesc - Play-by-play description of AB.
* atBatEvent - Event type result of the atBat, various types can be found here.
* hasOut - Binary, 1 if runner is out on play.
* pitcherTeamId - Unique identifier for pitching team.
* isPitcherHome - Binary, 1 if pitcher is home team.
* pitcherTeam - team name of pitching team.
* hitterTeamId - Unique identifier for hitting team.
* hitterTeam - Team name of hitting team.
* pitcherId
* pitcherName
* isStarter - Binary, 1 if player was game’s starting pitcher.
* pitcherHand - Player throw hand: “L”,”R”.
* hitterId
* hitterName
* batSide - Player bat side: “L”,”R”.
* pitchNumber - Pitch sequence number of AB.
* balls - Ball count after the event.
* strikes - Strike count after the event.
* isGB - Binary, 1 if at bat is a ground ball.
* isLD - Binary, 1 if at bat is a line drive.
* isFB - Binary, 1 if at bat is a fly ball.
* isPU - Binary, 1 if at bat is a pop up.
* launchSpeed - Measured speed of the hit ball.
* launchAngle - Vertical angle relative to the horizon at which the hit was launched.
* totalDistance - Total distance ball traveled.
* event - Events that may occur during an at-bat: strikeout, groundout, substitutions, stolen base, wild pitch, game advisory, etc.
* description - Text description of event.
* rbi - Number of runs batted in during AB.
* pitchType - Pitch type classification code, various types can be found here.
* call - Pitch or throw result classification code, various types can be found here.
* outs - Current/Final outs of AB.
* inPlay - True/False if ball was hit into play.
* isPaOver - Binary, 1 if this event is the end of the plate appearance.
* startSpeed - Speed in MPH of the ball at 50 feet in front of home plate.
* endSpeed - Speed in MPH of the ball as it crosses the front edge of home plate (0,0 in the x axis).
* nastyFactor - Evaluates several properties of each pitch, and rates the "nastiness" of the pitch on a scale from 0-100.
* breakAngle - Degrees clockwise (batter’s view) that the plane of the pitch deviates from the vertical.
* breakLength - Max distance that the pitch separates from the straight line between pitch start and pitch end.
* breakY - Distance from home plate where the break is greatest.
* spinRate - Rate of spin on the ball after it was released by pitcher in RPMs.
* spinDirection - The axis of rotation for the ball at release given as an angle that reflects how the spin will influence the ball trajectory. Pure back-spin is 180 degrees, pure side-spin that puts the ball to the 1b side is 90 degrees, pure-side spin that pulls the ball to the 3b side is 270 degrees, and pure top-spin is 0 or 360 degrees.
* pX - Horizontal position in feet of the ball as it crosses the front axis of home plate.
* pZ - Vertical position in feet above home plate of the ball as it crosses the front axis of home plate.
* aX - Ball acceleration on the z axis.
* aY - Ball acceleration on the y axis.
* aZ - Ball acceleration on the z axis.
* pfxX - Horizontal movement of the ball in inches.
* pfxZ - Vertical movement of the ball in inches.
* vX0 - Velocity of the ball from the x axis.
* vY0 - Velocity of the ball from the y axis; this is negative because 0,0,0 is behind the batter and the ball travels from pitcher mound towards 0,0,0.
* vZ0 - Velocity of the ball from the z axis.
* x - X coordinate where pitch crossed front of home plate.
* y - Y coordinate where pitch crossed front of home plate.
* x0 - Coordinate location of the ball at the point it was released from the pitchers hand on the x axis (time = 0).
* y0 - Coordinate location of the ball at the point it was released from the pitchers hand on the y axis (time = 0).
* z0 - Coordinate location of the ball at the point it was released from the pitchers hand on the z axis (time = 0).
* type - Type of event, either ‘pitch’ or ‘action’
* zone - Zone location number - see image below.

<div style="background-color:powderblue;">
    <h1><center>1.10 playerTwitterFollowers(column 10 of train.csv)</center></h1>
</div>
Nested JSON containing some players’ number of Twitter followers on that day.

In [ ]:
tmpdf = exshow("playerTwitterFollowers",0) 
tmpdf.head(3)

Twitter following data was collected by MLB from Twitter APIs for Major League players, on the first of each month, dating back to 1/1/2018. This dataset is not exhaustive of all players over all months, as not every player has/had a Twitter account, players may create/delete/reinstate accounts at random, or other scenarios preventing follower data from being collected on a given date.

* date - Date of follower count.
* playerId - Unique identifier for a player.
* playerName
* accountName - Name on player twitter account.
* twitterHandle - Player twitter handle.
* numberOfFollowers - Number of followers


<div style="background-color:powderblue;">
    <h1><center>1.11 teamTwitterFollowers(column 11 of train.csv)</center></h1>
</div>
 Nested JSON containing each team’s number of Twitter followers on that day.

In [ ]:
tmpdf = exshow("teamTwitterFollowers",0) 
tmpdf.head(3)

Twitter following data was collected by MLB from Twitter APIs for all 30 Major League teams, on the first of each month, dating back to 1/1/2018.

* date - Date of follower count.
* teamId - Unique identifier for a team.
* teamName
* accountName - Name on team twitter account.
* twitterHandle - Team twitter handle.
* numberOfFollowers - Number of followers.


<div style="background-color:powderblue;">
    <h1><center>teams.csv</center></h1>
</div>

Library containing high level information about all MLB teams.


In [ ]:
teams.head()

In [ ]:
print('teams Columns:')
teams.columns

In [ ]:
import plotly.express as px

fig = px.bar(teams.columns[:-1] , x=teams.nunique(), y=teams.columns, orientation='h')

fig.update_layout(title='Feature cardinality in teams set',
                  xaxis_title='unique values',
                  yaxis_title='features',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=1000)
fig.show()

> ### 
<div style="background-color:powderblue;">
    <h1><center>Players.csv</center></h1>
</div>

Library containing high level information about all MLB players in this dataset.

In [ ]:
players.head()

In [ ]:
print('Players Columns:')
players.columns

In [ ]:
fig = px.bar(players.columns[:-1] , x=players.nunique(), y=players.columns, orientation='h')

fig.update_layout(title='Feature cardinality in players set',
                  xaxis_title='unique values',
                  yaxis_title='features',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=1000)
fig.show()

In [ ]:
import missingno as msno
fig1 = msno.bar(players)
font = {'family': 'serif',
                'color':  'darkred',
                'weight': 'normal',
                'size': 30
        }
plt.title(label='Bar chart representation of the null values',fontdict=font)
plt.show()

In [ ]:
msno.matrix(players)
font = {'family': 'serif',
                'color':  'darkred',
                'weight': 'normal',
                'size': 30
        }
plt.title(label='Null values matrix of the dataset',fontdict=font)
plt.show()


<div style="background-color:powderblue;">
    <h1><center>Awards.csv</center></h1>
</div>

This file has awards won by players in the training set prior to the beginning of the daily data (i.e. before 2018).

* awardDate - Date award was given.
* awardSeason - Season award was from.
* awardId
* awardName
* playerId - Unique identifier for a player.
* playerName
* awardPlayerTeamId

In [ ]:
awards.head()

In [ ]:
fig = px.bar(awards.columns[:-1] , x=awards.nunique(), y=awards.columns, orientation='h')

fig.update_layout(title='Feature cardinality in awards set',
                  xaxis_title='unique values',
                  yaxis_title='features',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=1000)
fig.show()


<div style="background-color:powderblue;">
    <h1><center>Data merger</center></h1>
</div>

Refrence: [https://www.kaggle.com/columbia2131/mlb-lightgbm-starter-dataset-code-en-ja](https://www.kaggle.com/columbia2131/mlb-lightgbm-starter-dataset-code-en-ja)

In [ ]:
from pathlib import Path
TRAIN_DIR = Path('../input/mlb-pdef-train-dataset')
rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
scores = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')

In [ ]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching','lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date']

feature_cols = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching','baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']

In [ ]:
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')

# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)

<div style="background-color:powderblue;">
    <h1><center>Building Model</center></h1>
</div>

In [ ]:
X = train[feature_cols]
y = train[['target1', 'target2', 'target3', 'target4']]

In [ ]:
_index = (train['date'] < 20210401)
x_train = X.loc[_index].reset_index(drop=True)
y_train = y.loc[_index].reset_index(drop=True)
x_valid = X.loc[~_index].reset_index(drop=True)
y_valid = y.loc[~_index].reset_index(drop=True)

In [ ]:
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor

def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = LGBMRegressor(**params)
    model.fit(x_train, y_train, 
        eval_set=[(x_valid, y_valid)],  
        early_stopping_rounds=100, 
        verbose=400)
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('mae:', score)
    return oof_pred, model, score


# training lightgbm
params = {
 'objective':'mae',
 'reg_alpha': 2.361501631985262,
 'reg_lambda': 2.4000071578349456, 
 'n_estimators': 100000,
 'learning_rate': 0.0011097822790813784,
 'random_state': 42,
}

oof1, model1, score1 = fit_lgbm(x_train, y_train['target1'],x_valid, y_valid['target1'],params)
oof2, model2, score2 = fit_lgbm(x_train, y_train['target2'],x_valid, y_valid['target2'],params)
oof3, model3, score3 = fit_lgbm(x_train, y_train['target3'],x_valid, y_valid['target3'],params)
oof4, model4, score4 = fit_lgbm(x_train, y_train['target4'],x_valid, y_valid['target4'],params)

score = (score1+score2+score3+score4) / 4
print(f'score: {score}')


In [ ]:

players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']

null = np.nan
true = True
false = False

import mlb

env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    
    # creat dataset
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
    if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
        test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
    else:
        test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in scores.columns:
            if col == 'playerId': continue
            test_scores[col] = np.nan
    test_scores = test_scores.groupby('playerId').sum().reset_index()
    test = sample_prediction_df[['playerId']].copy()
    test = test.merge(players[players_cols], on='playerId', how='left')
    test = test.merge(test_rosters[rosters_cols], on='playerId', how='left')
    test = test.merge(test_scores[scores_cols], on='playerId', how='left')

    test['label_playerId'] = test['playerId'].map(player2num)
    test['label_primaryPositionName'] = test['primaryPositionName'].map(position2num)
    test['label_teamId'] = test['teamId'].map(teamid2num)
    test['label_status'] = test['status'].map(status2num)
    
    test_X = test[feature_cols]
    
    # predict
    pred1 = model1.predict(test_X)
    pred2 = model2.predict(test_X)
    pred3 = model3.predict(test_X)
    pred4 = model4.predict(test_X)
    
    # merge submission
    sample_prediction_df['target1'] = np.clip(pred1, 0, 100)
    sample_prediction_df['target2'] = np.clip(pred2, 0, 100)
    sample_prediction_df['target3'] = np.clip(pred3, 0, 100)
    sample_prediction_df['target4'] = np.clip(pred4, 0, 100)
    sample_prediction_df = sample_prediction_df.fillna(0.)
    del sample_prediction_df['playerId']
    
    env.predict(sample_prediction_df)

<div class="alert alert-block alert-info">
<h4>If you like this notebook, please upvote it! 
     Thank you! :)</h4>
</div>